# ASSIGNMENT 1

## Lara Monteserín Placer

## María Ferrero Medina

## Introduction

The aim of this project is to design a machine learning model that is able to predict the energy produced by the Sotavento wind farm. For this purpose, a dataset with 555 features ans 4748 instances is available.

The structure of the study will be the following:

1. Exploratory Data Analysis
2. Methodology
3. KNN model
4. Decission tree model
5. Ensemble method 1 model
6. Ensemble method 2 model
7. Selection and performance of the final model

For each of the models created, several steps have been followed to optimize them. First of all, a simple version of each model is created, with hyperparameters that seem reasonable, no feature selection and a basic imputation technique. Then, sequentially, models are improved.

1. First model
2. Feature selection
3. Imputation techniques
4. Hyperparameter tuning


Things to add: - another idea (new library?, new idea...)


## 1. Exploratory Data Analysis

Before starting to build the model, an EDA is made as a first approach to gain understanding of the dataset. In this Exploratory Data Analysis the data type of the features will be verified, the number of instances and features will be determined. Also, a brief summary of the missing values and columns with constant value will be included. 

### 1.1. Number of instances and features

This dataset has 4748 instances and 555 features.

### 1.2. Nature of the variables

This dataset contains information about the meteorological conditions in several locations, the time the measures of these conditions were made and the energy produced at each moment. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Read the data that is compressed as a gzip
wind_ava = pd.read_csv('wind_available.csv.gzip', compression="gzip")

# Display the first rows of the dataset just to see it
wind_ava.head()

# Display the data type of each column
column_data_types = wind_ava.dtypes
print(column_data_types)

energy     float64
year         int64
month        int64
day          int64
hour         int64
            ...   
v100.21    float64
v100.22    float64
v100.23    float64
v100.24    float64
v100.25    float64
Length: 555, dtype: object


After having checked the data types of all the different features, it has been verified that there are:

- 551 numerical variables (real numbers). From this 551, one is the energy, that is the output of the problem. And the remaining 550 are relative to the 22 different meteorological conditions measured at the 25 different locations.

- 4 numerical variables (integers). These 4 variables are the year, day, month and hour of the day. These variables characterize the moment the measures were taken.

### 1.3. Check for missing values 

In [2]:
# Return the number of Null values for each column
null_values = wind_ava.isnull().sum()
# Return the number of NaN values for each column (just in case they are not the same)
nan_values = wind_ava.isna().sum()

# Store in missing values the amount of Null and NaN values of each column
missing_values = pd.DataFrame({
    'Column': null_values.index,
    'Null Values': null_values.values,
    'NaN Values': nan_values.values
})

# Print the amount of Null and Nan values
print(missing_values)

# Identify columns with Null or NaN values
columns_with_null = wind_ava.columns[wind_ava.isnull().any()]
columns_with_nan = wind_ava.columns[wind_ava.isna().any()]

# Display the number of columns which have missing values
print("Number of columns with Null Values:", len(columns_with_null))
print("Number of columns with NaN Values:", len(columns_with_nan))

      Column  Null Values  NaN Values
0     energy            0           0
1       year            0           0
2      month            0           0
3        day            0           0
4       hour            0           0
..       ...          ...         ...
550  v100.21          261         261
551  v100.22          387         387
552  v100.23          569         569
553  v100.24          579         579
554  v100.25          436         436

[555 rows x 3 columns]
Number of columns with Null Values: 550
Number of columns with NaN Values: 550


All meteorological variables have missing values in different instances. The 4 categories that characterize the moment the measure was made and the target feature'energy' do not have missing values.


### 1.4. Check for constant columns


In [ ]:
# Check for constant values in each column
constant_columns = wind_ava.columns[wind_ava.nunique() == 1]

# Print columns with constant values
print("Columns with constant values:", constant_columns)


There are no constant columns. 


### 1.5. Type of problem

The objective of the model is to estimate the energy, as it is a continuous numerical value, this is a **regression problem**. 

## 2. Methodology

This section will explain the methodology that is going to be followed to evaluate the models. The evaluation techniques that will be used for outer evaluation and inner evaluation. And also the metrics that will determine the future performance of the model.

- On the one hand, for the **inner evaluation**, crossvalidation will be the method applied. Crossvalidation will be used to determine which is the best combination of hyperparameters. 

- On the other hand, for the **outer evaluation**, holdout evaluation will be used. Thie method will be used to estimate the future performance of the designed method.

Later, to improve the performance of the method, once it is already computed the outer performance, the hyperparameters will be tuned again but this time using the whole dataset.

The objective function that is going to be used for the validation of the method is the Mean Squared Error (MSE). This metric is more sensitive to outliers and to distant values as it squares the magnitudes. This is useful to penalize the errors that are larger, and avoid having a model that might have such large errors. 

**It could be longer this explanation**

Note: as the variable that is going to be predicted is the 'energy', only the variables related to the meteorological characteristics will be used. It is considered that the energy produced does not depend on the moment of the day it is being produced.

## 3. KNN Regressor

First, the KNN algorithm is used for the predictions.

### 3.1. First model

A first model using the KNN algorithm will be implemented. It will use the default hyperparameters, the KNN Imputer as imputation strategy and no feature selection. This model will be updated and improved but it is just a simple approach to compare the adjusted models.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import time

#Here, we define a generic random seed that will be used along the assignment
rs = 100515585 

# First, data will be divided into train and test set 
# Considering it is a time series, it must be split in an appropiate way
wind_ava['timestamp'] = pd.to_datetime(wind_ava[['year', 'month', 'day', 'hour']])
wind_ava = wind_ava.sort_values(by='timestamp')

train_size = 0.8 
split_index = int(len(wind_ava) * train_size)
wind_ava = wind_ava.drop(columns=['timestamp'])

# Divide the data into X_train, y_train, X_test, y_test
train_data = wind_ava.iloc[:split_index]
test_data = wind_ava.iloc[split_index:]

X_train = train_data.drop('energy', axis=1)
y_train = train_data['energy']
X_test = test_data.drop('energy', axis=1)
y_test = test_data['energy']

# Now the first model will be created
first_knn = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',KNeighborsRegressor())])
first_knn.fit(X_train,y_train)
y_predicted = first_knn.predict(X_test)
MSE = mean_squared_error(y_test,y_predicted)

print('MSE: ', MSE)


MSE:  398902.9111835073


### 3.2 Feature selection

Now, feature selection will be used in order to remove variables that are not affecting directly the target. This is done manually, selecting different feature combinations that seem to be more accurate for the problem. Three different cases have been considered:
- FIRST OPTION. Selecting only the features related to the location of the wind farm (Sotavento). This is the features that contain the sufix 13.
- SECOND OPTION. Selecting only the features related to the wind characteristics. This is the features that start with u or v (the vertical and horizontal components of the wind).
- THIRD OPTION. Selecting only the features which are both located in Sotavento (location 13) and related to the wind characteristics. This is the features that start with u or v and end with sufix 13.

This strategy will be followed for the feature selection of the rest of the models. This is sections 4.2, 5.2 and 6.2.

In [4]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)
X_1 = wind_ava.filter(regex='\.13$', axis=1)

# Selecting the target
y_1 = wind_ava['energy']

train_size = 0.8
split_index = int(len(X_1) * train_size)

# Divide the data into X_train, y_train, X_test, y_test
X_train_1 = X_1.iloc[:split_index]
X_test_1 = X_1.iloc[split_index:]
y_train_1 = y_1.iloc[:split_index]
y_test_1 = y_1.iloc[split_index:]

# Now the first model will be created
first_knn = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',KNeighborsRegressor())])
first_knn.fit(X_train_1,y_train_1)
y_predicted_1 = first_knn.predict(X_test_1)
MSE = mean_squared_error(y_test_1,y_predicted_1)

print('MSE: ', MSE)

MSE:  442428.52588486316


In [5]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)
X_2 = wind_ava.filter(regex='^(u|v).*$', axis=1)

# Split into train and test sets
y_2 = wind_ava['energy']

train_size = 0.8 
split_index = int(len(X_2) * train_size)

# Divide the data into X_train, y_train, X_test, y_test
X_train_2 = X_2.iloc[:split_index]
X_test_2 = X_2.iloc[split_index:]
y_train_2 = y_2.iloc[:split_index]
y_test_2 = y_2.iloc[split_index:]

# Define the first model as a Pipeline with a preprocessing stage with knn
second_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',KNeighborsRegressor())])
second_tree.fit(X_train_2,y_train_2)
y_predicted_2 = second_tree.predict(X_test_2)
MSE_2 = mean_squared_error(y_test_2,y_predicted_2)

print('MSE: ', MSE_2)

MSE:  178196.27234742316


In [6]:
from sklearn.metrics import mean_squared_error

# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento
X_3 = wind_ava.filter(regex='^(u|v).*\.13$', axis=1)

# Split into train and test sets
y_3 = wind_ava['energy']

train_size = 0.8  # Porcentaje de datos para entrenamiento
split_index = int(len(X_3) * train_size)

# Divide the data into X_train, y_train, X_test, y_test
X_train_3 = X_3.iloc[:split_index]
X_test_3 = X_3.iloc[split_index:]
y_train_3 = y_3.iloc[:split_index]
y_test_3 = y_3.iloc[split_index:]

# Define the first model as a Pipeline with a preprocessing stage with knn
third_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',KNeighborsRegressor())])
third_tree.fit(X_train_3,y_train_3)
y_predicted_3 = third_tree.predict(X_test_3)
MSE_3 = mean_squared_error(y_test_3,y_predicted_3)

print('MSE: ', MSE_3)

MSE:  191894.0329901347


Results above show that the best feature selection strategy is the second, selecting only the features related to the wind. It improves the performance of the model as it is not considering features that are not affecting the target and introduce unnecessary noise and lead to overfitting.

From this moment, the KNN model will be built using only the features related to the wind (that is X_train_2, y_train_2, X_test_2, y_test_2).

### 3.3. Imputation techniques

Once the feature selection has been implemented, it is time to choose the imputation technique that is going to be used. 

Two different strategies will be considered, that belong to two different types of techniques:

- Univariate imputation. That only uses the values from the feature that is going to be imputed. For this it will be used the Simple Imputer, that imputes all the missing values with the mean of the feature. We have chosen the mean instead of the median because there are not many outliers that could affect the distribution of the data.


- Multivariate imputation. Which uses values from the feature that is going to be imputed and also from other features. Two multivariate imputation techniques are used:
    - KNN Imputer. Is based on the KNN algorithm.
    - Iterative Imputer. Is based on iterative models that compute the values for the missing categories.

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer

# FIRST OPTION: one univariate technique will be used for imputation, this is done with the Simple Imputer, using the mean
simple_knn = Pipeline([('imputer',SimpleImputer(strategy = 'mean')),('regression',KNeighborsRegressor())])

simple_knn.fit(X_train_2,y_train_2)
y_predicted_2 = simple_knn.predict(X_test_2)
MSE_3 = mean_squared_error(y_test_2,y_predicted_2)

print('MSE with Simple Imputer: ', MSE_3)

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer

# SECOND OPTION: multivariate technique. KNN Imputer using 3 neighbors

knn_knn = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',KNeighborsRegressor())])

knn_knn.fit(X_train_2,y_train_2)
y_predicted_2 = knn_knn.predict(X_test_2)
MSE_3 = mean_squared_error(y_test_2,y_predicted_2)

print('MSE with : ', MSE_3)

# Result: 178196

In [ ]:
# Update the version of scikit-learn if it returns an Error with IterativeImputer
!pip install --upgrade scikit-learn --user

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# THIRD OPTION: multivariate technique. KNN Imputer using 3 neighbors

iterative_knn = Pipeline([('imputer',IterativeImputer(max_iter = 14, random_state = rs)),('regression',KNeighborsRegressor())])

iterative_knn.fit(X_train_2,y_train_2)
y_predicted_2 = iterative_knn.predict(X_test_2)
MSE_3 = mean_squared_error(y_test_2,y_predicted_2)

print('MSE with : ', MSE_3)

# Result: 174930

From the results above, it is demonstrated that the simple imputer is the worst in terms of performance with respect to the Mean Squared Error.

On the other hand, the multivariate techniques, KNN Imputer and the Iterative Imputer give similar results, but the Iterative Imputer has a lower Mean Squared Error.

From this point on, the imputation technique will be the Iterative Imputation, so the pipeline will be knn_knn.

### 3.4. Hyperparameter tuning

Now we will perform a crucial stage, that is to optimize the model by performing hyperparameter tuning. In the case of KNN algorithm, the main hyperparameter is the number of neighbors.

Several approaches for hyperparameter tuning will be implemented. The optimization time will be measured and also the best combination of hyperparameters for each method will be compared to determine which HPO technique is the most suitable. 

For HPO, Pipelines will not be used in order to focus on the critical hyperparameters of each step. The idea is to get the most effective optimization. First, the function to optimize will be created and so will the parameter grid. Also, the imputation will be made for the training and testing data. This will be done first to save execution time on each occasion the HPO is done. 

In [ ]:
# THIS TAKES ~40s
# IMPUTATION

# Perform the imputation for training data (outside the pipeline)
imputer = IterativeImputer(max_iter=14, random_state=rs)
X_train_imputed = imputer.fit_transform(X_train_2)

# Imputate also for the test set to estimate future performance
X_test_imputed = imputer.fit_transform(X_test_2)


In [ ]:
# CREATE THE MODEL TO OPTIMIZE

# Create the regressor with no hyperparamters defined
knn_default = KNeighborsRegressor()

# Create a dictionary with the values for the hyperparameters
params = {'n_neighbors': np.arange(1, 31)}

In [ ]:
# FIRST STRATEGY - Randomized Search

# Import required libraries
from sklearn.model_selection import RandomizedSearchCV

# Create a randomized search instance
random_search = RandomizedSearchCV(
    knn_default,
    param_distributions=params,
    n_iter=10,  # Number of iterations (can be adjusted)
    cv=5,       # Number of cross-validation folds
    scoring='neg_mean_squared_error',  # Mean squared error is the error metric
    random_state=rs
)

# Train the model, optimize the hyperparaneters and measure the time
t_0 = time.time()
random_search.fit(X_train_imputed, y_train_2)
t_1 = time.time()

# Estimation of future performance
y_test_pred = random_search.predict(X_test_imputed)

print("KNN's MSE with HPO: ",
     mean_squared_error(y_test_2, y_test_pred))

print("Best Hyperparameters: ", random_search.best_params_)

# print("MSE inner evaluation: ", mse_future)
print("Execution times: ",t_1-t_0)

In [ ]:
# Poniendo n_neighbors = trial.suggest_int('n_neighbors', 1, 3) tarda 970 segundazos
# TAKES LONG TO EXECUTE !-!-!-!

import optuna
from sklearn.model_selection import cross_val_score, KFold

# SECOND STRATEGY - Optuna

# Define the objective function
def objective(trial):
    # Hyperparameters that are to be tuned
    n_neighbors = trial.suggest_int('n_neighbors', 1, 31)
    params = {'n_neighbors': n_neighbors}
    
    # Estimator with suggested hyperparameters
    knn_default = KNeighborsRegressor()
    
    # Define the neg means squared error as the score and the inner evaluation as corssvalidation 
    inner_score = cross_val_score(iterative_knn, X_train_imputed, y_train_2, cv=5, scoring='neg_mean_absolute_error').mean()
    return inner_score

# Train the model and perform HPO
# Measure the time
sampler = optuna.samplers.TPESampler(seed=rs)
study = optuna.create_study(direction='maximize',sampler=sampler)

iterations = 30
t_0 = time.time()
study.optimize(objective, n_trials = iterations)
t_1 = time.time()


print('5: ', time.time())

# Best hyperparameters
best_params = study.best_params

print('6: ', time.time())

regr_opt = KNeighborsRegressor(**best_params)


print('7: ', time.time())

regr_opt.fit(X_train_imputed,y_train_2)

print('8: ', time.time())

# Estimate future performance
y_test_pred = regr_opt.predict(X_test_imputed)
mse_future = mean_squared_error(y_test_pred, y_test_2)

# Print results
# Print the results for the best combination
print("MSE future performance: ", mse_future)
print("Best Hyperparameters: ", best_params)
print("Execution time: ",t_1-t_0)

In [ ]:
# THIS TAKES A LONG TIME !-!-!

# THIRD STRATEGY - Successive halving

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV


# Create
halving = HalvingGridSearchCV(knn_default,
                              params,
                              scoring= 'neg_mean_squared_error',
                              cv=5,
                              random_state=rs,
                              factor=2,
                              min_resources='exhaust',
                              max_resources='auto',
                              n_jobs=-1, verbose=1)

halving.fit(X_train_imputed, y_train_2)

# Estimation of future performance
# Measure the ex time
t_0=time.time()
y_test_pred = halving.predict(X_test_imputed)
t_1=time.time()

print("KNN's MSE with HPO: ",
     mean_squared_error(y_test_2, y_test_pred))
print("best_params", halving.best_params_)
print("Execution times: ",t_1-t_0)

### 3.5. Conclusions about KNN model

- Hyperparameter tuning: the fastest method and the one that reaches the optimal solution
- MSE

Out of all these trials, we will now use the best model obtained from this study in order to compare it with the remaining models.

## 4. Decision Tree

### 4.1. First model

A first model using the Decision Tree algorithm will be implemented. It will use the default hyperparameters, the KNN Imputer as imputation strategy and no feature selection. This model will be updated and improved but it is just a simple approach to compare the adjusted models.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import r2_score
import pandas as pd

# FIRST OPTION - Univariate technique 
# Define the first tree as a Pipeline with a preprocessing stage with decision tree
first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor(random_state=rs))])
first_tree.fit(X_train,y_train)
y_predicted = first_tree.predict(X_test)

MSE = mean_squared_error(y_test,y_predicted)

print('MSE: ', MSE)

#284418

MSE:  284418.83268557896


### 4.2. Feature selection

For the feature selection, as in the case before, three different cases have been considered:
- FIRST OPTION. Selecting only the features related to the location of the wind farm (Sotavento). This is the features that contain the sufix 13.
- SECOND OPTION. Selecting only the features related to the wind characteristics. This is the features that start with u or v (the vertical and horizontal components of the wind).
- THIRD OPTION. Selecting only the features which are both located in Sotavento (location 13) and related to the wind characteristics. This is the features that start with u or v and end with sufix 13.

In [10]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)
X_1 = wind_ava.filter(regex='\.13$', axis=1)

# Split into train and test sets
y_1 = wind_ava['energy']

train_size = 0.8  # Porcentaje de datos para entrenamiento
split_index = int(len(X_1) * train_size)

# Divide the data into X_train, y_train, X_test, y_test
X_train_1 = X_1.iloc[:split_index]
X_test_1 = X_1.iloc[split_index:]
y_train_1 = y_1.iloc[:split_index]
y_test_1 = y_1.iloc[split_index:]

# Define the first model as a Pipeline with a preprocessing stage with knn
# first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('feature_selection',SelectKBest(f_regression)),('regression',DecisionTreeRegressor())])
first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor(random_state=rs))])
first_tree.fit(X_train_1,y_train_1)
y_predicted_1 = first_tree.predict(X_test_1)
MSE_1 = mean_squared_error(y_test_1,y_predicted_1)

print('MSE: ', MSE_1)

MSE:  320021.95956105256


In [11]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)
X_2 = wind_ava.filter(regex='^(u|v).*$', axis=1)

# Split into train and test sets
y_2 = wind_ava['energy']

train_size = 0.8
split_index = int(len(X_2) * train_size)

# Divide the data into X_train, y_train, X_test, y_test
X_train_2 = X_2.iloc[:split_index]
X_test_2 = X_2.iloc[split_index:]
y_train_2 = y_2.iloc[:split_index]
y_test_2 = y_2.iloc[split_index:]

# Define the first model as a Pipeline with a preprocessing stage with knn
second_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor(random_state=rs))])
second_tree.fit(X_train_2,y_train_2)
y_predicted_2 = second_tree.predict(X_test_2)
MSE_2 = mean_squared_error(y_test_2,y_predicted_2)

print('MSE: ', MSE_2)

MSE:  269120.25575199997


In [12]:
from sklearn.metrics import mean_squared_error

# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento
X_3 = wind_ava.filter(regex='^(u|v).*\.13$', axis=1)

# Split into train and test sets
y_3 = wind_ava['energy']

train_size = 0.8  # Porcentaje de datos para entrenamiento
split_index = int(len(X_3) * train_size)

# Divide the data into X_train, y_train, X_test, y_test
X_train_3 = X_3.iloc[:split_index]
X_test_3 = X_3.iloc[split_index:]
y_train_3 = y_3.iloc[:split_index]
y_test_3 = y_3.iloc[split_index:]

# Define the first model as a Pipeline with a preprocessing stage with knn
third_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor(random_state=rs))])
third_tree.fit(X_train_3,y_train_3)
y_predicted_3 = third_tree.predict(X_test_3)
MSE_3 = mean_squared_error(y_test_3,y_predicted_3)

print('MSE: ', MSE_3)

MSE:  298762.4396745263


From the previous results, we can see that the best feature selection is the second, selecting only the features related to the wind.

Based on this, the Decision Tree model will be built using features related to the wind (that is X_train_2, y_train_2, X_test_2, y_test_2).

The best MSE achieved until this point is: 269120.25

### 4.3. Imputation techniques

Now it is time to choose the imputation technique that is going to be used. Authomatic techniques will be used (instead of manual imputation).

Two types of techniques will be considered:
- Univariate imputation. That only uses the values from the feature that is going to be imputed. For this it will be used the Simple Imputer, that imputes all the missing values with the mean of the feature. We have chosen the mean instead of the median because there are not many outliers that could affect the distribution of the data.

- Multivariate imputation. Which also uses values from other features. Two multivariate imputation techniques are used:
    - KNN Imputer. Is based on the KNN algorithm.
    - Iterative Imputer. Is based on iterative models that compute the values for the missing categories.

In [13]:
from sklearn.impute import KNNImputer, SimpleImputer

# FIRST: one univariate technique will be used for imputation, this is done with the Simple Imputer, using the mean

simple_tree = Pipeline([('imputer',SimpleImputer(strategy = 'mean')),('regression',DecisionTreeRegressor(random_state=rs))])

simple_tree.fit(X_train_2,y_train_2)
y_predicted = simple_tree.predict(X_test_2)
MSE = mean_squared_error(y_test_2,y_predicted)

print('MSE : ', MSE)

MSE :  294190.42834821047


In [14]:
from sklearn.impute import KNNImputer, SimpleImputer

# SECOND: multivariate technique. KNN Imputer using 3 neighbors

knn_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor(random_state=rs))])

knn_tree.fit(X_train_2,y_train_2)
y_predicted = knn_tree.predict(X_test_2)
MSE = mean_squared_error(y_test_2,y_predicted)

print('MSE : ', MSE)

# 280817.7
# 281559.36
# Why does MSE change???

MSE :  269120.25575199997


In [15]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterative_tree = Pipeline([('imputer',IterativeImputer(max_iter = 14, random_state = rs)),('regression',DecisionTreeRegressor(random_state=rs))])

iterative_tree.fit(X_train_2,y_train_2)
y_predicted = iterative_tree.predict(X_test_2)
MSE = mean_squared_error(y_test_2,y_predicted)

print('MSE : ', MSE)

MSE :  255083.116514


From the results above, we can see that the simple imputer is the worst in terms of performance with respect to the Mean Squared Error.

On the other hand, the multivariate techniques: KNN Imputer and the Iterative Imputer give similar results, but the Iterative Imputer has a lower Mean Squared Error. From this point on, the imputation technique will be the Iterative Imputation, so the pipeline will be knn_tree.

### 4.4. Hyperparameter tuning

Once the imputation and feature selection have been performed, it is time to improve the performance of the classifier. Hyperparameter tuning will be carried out using crossvalidation as the inner evaluation method. 

In [16]:
# IMPUTATION

# Perform the imputation for training data (outside the pipeline)
imputer = IterativeImputer(max_iter=14, random_state=rs)
X_train_imputed = imputer.fit_transform(X_train_2)

# Imputate also for the test set to estimate future performance
X_test_imputed = imputer.fit_transform(X_test_2)

In [18]:
# CREATE THE MODEL TO OPTIMIZE
from sklearn.model_selection import KFold

# Create the regressor with no hyperparamters defined
tree_default = DecisionTreeRegressor()

# Create a dictionary with the values for the hyperparameters
# Define the grid with all possible values for the hyperparameters
# Define the number of folds and the conditions for the crossvalidation
param_grid = {'max_depth': list(range(2,16,2)), 'min_samples_split':list(range(2,16,2))}
k_folds = KFold(n_splits = 5, shuffle = True, random_state =rs)


In [20]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import metrics
import time

# FIRST STRATEGY: GridSearch 

# Measure the time it takes to perform the hyperparameter tuning
t_0 = time.time()
# Perform the hyperparameter tuning using the previously defined k_folds for crossvalidation
tuning = GridSearchCV(
    tree_default,
    param_grid,
    scoring='neg_mean_squared_error',
    cv=k_folds,
    n_jobs=1,verbose=1)

# Fit the models (all possible combinations)
tuning.fit(X_train_imputed,y_train_2)
t_1 = time.time()

# Access the best estimator
best_model = tuning.best_estimator_

# Access the best hyperparameters
best_params = tuning.best_params_

# Access the cross-validated results
cv_results = tuning.cv_results_

# Print MSE for each fold
for i in range(5):
    fold_mse = -cv_results[f"split{i}_test_score"][tuning.best_index_]
    print(f"MSE for Fold {i+1}: {fold_mse}")

# Calculate and print the mean MSE across folds
mean_mse = -cv_results["mean_test_score"][tuning.best_index_]


# Print the results for the best combination
print("Mean MSE across Folds: ", mean_mse)
print("Best Hyperparameters: ", best_params)
print("Execution time: ",t_1-t_0)

# ESTIMATE FUTURE PERFORMANCE



Fitting 5 folds for each of 49 candidates, totalling 245 fits
MSE for Fold 1: 163717.66342913502
MSE for Fold 2: 144317.58109068763
MSE for Fold 3: 167130.39533087678
MSE for Fold 4: 168761.23816298658
MSE for Fold 5: 164268.05666073156
Mean MSE across Folds:  161638.98693488352
Best Hyperparameters:  {'max_depth': 6, 'min_samples_split': 12}
Execution time:  92.45113515853882


In [23]:
# Estimation of future performance
y_test_pred = tuning.predict(X_test_imputed)

print("KNN's MSE with HPO: ",
     mean_squared_error(y_test_2, y_test_pred))


KNN's MSE with HPO:  189639.60576969493


In [ ]:
# Install OPTUNA if required
!pip install optuna

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score, KFold

# SECOND STRATEGY: Optuna

# Define the objective function
def objective(trial):
    # Hyperparameters that are going to be tuned
    max_depth = trial.suggest_int('max_depth',2,16)
    min_samples_split = trial.suggest_int('min_samples_split',2,16)
    
    # Estimator with suggested hyperparameters
    params = {'max_depth': max_depth, 'min_samples_split':min_samples_split}
    regr_tree = DecisionTreeRegressor(random_state = 100515585, **params)
    
    # Define the neg means squared error as the score and the inner evaluation as corssvalidation 
    inner_score = cross_val_score(regr_tree,X_train_2,y_train_2,cv=k_folds,scoring='neg_mean_absolute_error').mean()
    return inner_score

# Train the model and perform HPO
# Measure the time
t_0 = time.time()
sampler = optuna.samplers.TPESampler(seed=rs)
study = optuna.create_study(direction='maximize',sampler=sampler)
iterations = 30
study.optimize(objective, n_trials = iterations)
t_1 = time.time()

# Best hyperparameters
best_params = study.best_params
regr_opt = DecisionTreeRegressor(random_state = rs, **best_params)
regr_opt.fit(X_train_2,y_train_2)

# ESTIMATE FUTURE PERFORMANCE
y_test_pred = regr_opt.predict(X_test_2)
mse_future = mean_squared_error(y_test_pred, y_test_2)

# Print results
# Print the results for the best combination
print("MSE future performance: ", mse_future)
print("Best Hyperparameters: ", best_params)
print("Execution time: ",t_1-t_0)

In [ ]:
#Successive Halving

!pip install optuna scikit-optimize

import optuna
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import time

# Define the objective function
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 2, 16)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 16)
    params = {'max_depth': max_depth, 'min_samples_split': min_samples_split}
    regr_tree = DecisionTreeRegressor(random_state=100515585, **params)
    inner_score = cross_val_score(regr_tree, X_train_2, y_train_2, cv=k_folds, scoring='neg_mean_absolute_error').mean()
    return inner_score

if __name__ == "__main__":
    t_0 = time.time()

    # Use TPESampler for Bayesian optimization
    sampler = optuna.samplers.TPESampler(seed=100515585)
    study = optuna.create_study(direction='maximize', sampler=sampler)

    # Run optimization
    n_trials = 30
    for _ in range(n_trials):
        trial = study.ask()
        value = objective(trial)
        study.tell(trial, value)

    t_1 = time.time()

    best_params = study.best_params
    regr_opt = DecisionTreeRegressor(random_state=100515585, **best_params)
    regr_opt.fit(X_train_2, y_train_2)

    y_test_pred = regr_opt.predict(X_test_2)
    mse_future = mean_squared_error(y_test_pred, y_test_2)

    print("MSE future performance: ", mse_future)
    print("Best Hyperparameters: ", best_params)
    print("Execution time: ", t_1 - t_0)


## 5. Random Forest Regressor

### 5.1 First model


In [ ]:
from sklearn.ensemble import RandomForestRegressor
t_0 = time.time()
# Create the ensemble model pipeline
random_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
random_forest.fit(X_train, y_train)

# Predict on the test data
y_pred = random_forest.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

### 5.2 Feature selection


In [ ]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)
first_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
first_forest.fit(X_train_1, y_train_1)

# Predict on the test data
y_predicted_1_rf = first_forest.predict(X_test_1)

# Evaluate the model
MSE_1 = mean_squared_error(y_test_1, y_predicted_1_rf)
print("Mean Squared Error:", MSE_1)

In [ ]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)

second_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
second_forest.fit(X_train_2,y_train_2)

# Predict on the test data
y_predicted_2_rf = second_forest.predict(X_test_2)

# Evaluate the model
MSE_2 = mean_squared_error(y_test_2, y_predicted_2_rf)
print("Mean Squared Error:", MSE_1)


In [ ]:
# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento

third_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
third_forest.fit(X_train_3,y_train_3)

# Predict on the test data
y_predicted_3_rf = third_forest.predict(X_test_3)

# Evaluate the model
MSE_3 = mean_squared_error(y_test_3, y_predicted_3_rf)
print("Mean Squared Error:", MSE_3)


El error no mejora aplicando feature selection.

## 6. Bagging Regressor with KNN models
### 6.1 First model

In [ ]:
from sklearn.ensemble import BaggingRegressor

# Create the ensemble model pipeline
knn_ensemble_pipeline = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
knn_ensemble_pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = knn_ensemble_pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


### 6.2 Feature selection

In [ ]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)

first_ensemble = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
first_ensemble.fit(X_train_1, y_train_1)

# Predict on the test data
y_predicted_1_e = first_ensemble.predict(X_test_1)

# Evaluate the model
MSE_1 = mean_squared_error(y_test_1, y_predicted_1_e)
print("Mean Squared Error:", MSE_1)


In [ ]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)

second_ensemble = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
second_ensemble.fit(X_train_2, y_train_2)

# Predict on the test data
y_predicted_2_e = second_ensemble.predict(X_test_2)

# Evaluate the model
MSE_2 = mean_squared_error(y_test_2, y_predicted_2_e)
print("Mean Squared Error:", MSE_2)

In [ ]:
# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento

third_ensemble = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
third_ensemble.fit(X_train_3, y_train_3)

# Predict on the test data
y_predicted_3_e = third_ensemble.predict(X_test_3)

# Evaluate the model
MSE_3 = mean_squared_error(y_test_3, y_predicted_3_e)
print("Mean Squared Error:", MSE_3)

From the previous results, it is demonstrated that the error from the model improves applying feature selection. The best results are obtained selecting only the features related to the wind. The second best result is obtained selecting only the features from Sotavento and related to the wind.

Based on this results, for the KNN model, only the features related to the wind will be selected (second option). So, from now on, it will be used X_2 and y_2.